In [7]:
from engine import Engine
from core_components.entities.library import PlayerCharactor
from core_components.tiles.base import TileCoordinate, TileTuple
from core_components.atlas import Atlas
import tcod
import numpy as np

player = PlayerCharactor(color=(0,0,0), name="player")
expected_bits = np.full([80,50], fill_value=True)

game = Engine()
print(game.state.map.active.tiles['graphic'][0:2, 0:2])
game.state.map.active.set_state_bits('visible', expected_bits)
game.state.map.active.set_state_bits('seen', expected_bits)
game.state.map.active.update_state()
print(game.state.map.active.tiles['graphic'][0:2, 0:2])


c:\Users\jason\workspaces\repos\jrl\src\core_components\tiles\base.py:439: UserWarning: Cannot align corners without 'center', 'width', and 'height' attributes set.
  warn("Cannot align corners without 'center', 'width', and 'height' attributes set.", UserWarning)


[[(32, [255, 255, 255], [0, 0, 0]) (32, [255, 255, 255], [0, 0, 0])]
 [(32, [255, 255, 255], [0, 0, 0]) (32, [255, 255, 255], [0, 0, 0])]]
[[(32, [255, 255, 255], [130, 110,  50])
  (32, [255, 255, 255], [130, 110,  50])]
 [(32, [255, 255, 255], [130, 110,  50])
  (32, [255, 255, 255], [130, 110,  50])]]


In [12]:
game.state.map.active.areas['0'].get_random_location()

TileCoordinate(x=20, y=20, parent_map_size=([80], [50]))

In [ ]:
import numpy as np
expected_bits = np.full([80,50], fill_value=True)
a.active.set_state_bits('visible', expected_bits)
a.active.visible

In [ ]:
a.active.set_state_bits('seen', expected_bits)

In [ ]:
print(a.active.tiles['graphic'][0:2, 0:2])
a.active.update_state()
print(a.active.tiles['graphic'][0:2, 0:2])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(game.state.map.active.tiles['blocks_movement'])

game.state.map.active.tiles['blocks_vision'] = False



In [ ]:
game.state.roster.player = player
game.state.roster.player.location = TileCoordinate(TileTuple(([10],[10])))  
move_event = tcod.event.KeyDown(
        sym=tcod.event.KeySym.RIGHT,
        scancode=tcod.event.Scancode.RIGHT,
        mod=tcod.event.Modifier.NONE,
        repeat=False,
    )
game.start()

In [ ]:
import time


print(player.location)
game.state.events.put(move_event)
time.sleep(0.2)
print(player.location)

In [ ]:
game.state.map.active.tiles['graphic']

In [ ]:
from engine import Engine
from core_components.events.library import InputEvent, GameStartEvent, GameOverEvent
import tcod
import threading
import time


GAMESTART = GameStartEvent()
GAMEOVER = GameOverEvent('Game Over')
ESCAPE = tcod.event.KeyDown(sym=27, scancode=1, repeat=False, mod=0)

game = Engine()


In [ ]:
game.start()

In [ ]:
print(game.state.game_over.is_set(), game.state.events.qsize(), game.state.actions.qsize())

In [ ]:
game_event = GAMESTART
game.state.events.put(game_event)
time.sleep(0.2)
print(game.state.game_over.is_set(), game.state.events.qsize(), game.state.actions.qsize())

In [ ]:
game_event = ESCAPE
game.state.events.put(game_event)
time.sleep(0.2)
print(game.state.game_over.is_set(), game.state.events.qsize(), game.state.actions.qsize())

In [ ]:
import threading
import sys

def custom_hook(args):
    print(f"Thread failed: {args.thread.name}")
    print(f"Exception type: {args.exc_type}")
    print(f"Exception value: {args.exc_value}")

# Set the global exception hook
threading.excepthook = custom_hook

def faulty_worker():
    raise RuntimeError("Something went wrong!")

thread = threading.Thread(target=faulty_worker)
thread.start()
thread.join()
# Main thread continues after the hook is called
print("Main thread finished")

In [13]:
#!/usr/bin/env python3
import tcod
from engine import Engine
from core_components.entities.library import PlayerCharactor
from core_components.tiles.base import TileCoordinate, TileTuple
import numpy as np

def main() -> None:
    screen_width = 80
    screen_height = 50

    tileset = tcod.tileset.load_tilesheet(
        "dejavu10x10_gs_tc.png", 32, 8, tcod.tileset.CHARMAP_TCOD
    )
    
    game = Engine()
    player = PlayerCharactor(color=(255,255,255), symbol="@", name="player", location=TileCoordinate(TileTuple(([10],[10]))))
    game.state.roster.player = player
    starting_location = game.state.map.active.areas['0'].get_random_location()
    player.location = starting_location

    expected_bits = np.full([80,50], fill_value=True)
    # game.state.map.active.set_state_bits('visible', expected_bits)
    game.state.map.active.set_state_bits('seen', expected_bits)

    game.start()
    context = tcod.context.new(x = screen_width, y =screen_height, tileset=tileset,
                                         title="Yet Another Roguelike Tutorial", vsync=True)
    
    while True:
        # Update Console
        game.state.map.active.update_state()

        console = context.new_console(screen_width, screen_height, order="F")
        console.rgb[0:screen_width, 0:screen_height] = game.state.map.active.get_state()
        console.print(player.location.x, player.location.y, player.symbol, fg=player.color)
        context.present(console)
        console.clear()

        # Update Game State
        for game_event in tcod.event.wait():
            if isinstance(game_event, tcod.event.KeyDown):
                game.state.events.put(game_event)

        if game.state.game_over.is_set():
            context.close()   
            break
        
if __name__ == "__main__":
    main()

c:\Users\jason\workspaces\repos\jrl\src\core_components\tiles\base.py:439: UserWarning: Cannot align corners without 'center', 'width', and 'height' attributes set.
  warn("Cannot align corners without 'center', 'width', and 'height' attributes set.", UserWarning)
RENDER:Created renderer: direct3d11


Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F2ED0>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F29F0>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F26F0>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F2390>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F20F0>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F2990>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F2330>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F2E70>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F2510>
Action performed: <core_components.actions.library.EntityMoveAction object at 0x00000226B62F2270>
Action performed: <c

In [ ]:
from engine import Engine
from core_components.entities.library import PlayerCharactor
from core_components.tiles.base import TileCoordinate, TileTuple
player = PlayerCharactor(color=(0,0,0), name="player")
another_player = PlayerCharactor(color=(0,0,0), name="another_player")
game.state.roster.player.location = TileCoordinate(TileTuple(([10],[10])))  
game = Engine()